In [1]:
from dolfinx import default_scalar_type, fem, mesh
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import numpy as np
import ufl

In [2]:
Omega = mesh.create_interval(MPI.COMM_WORLD, 100, [0, 1])
V = fem.functionspace(Omega, ('Lagrange', 1))

In [3]:
dofs_L = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0))
dofs_R = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 1))
u_L = fem.Function(V)#fem.Constant(Omega, default_scalar_type(0))
u_L.interpolate(lambda x: x[0]*0)
u_R = fem.Function(V)#fem.Constant(Omega, default_scalar_type(10))
u_R.interpolate(lambda x: x[0]*0 + 10)
bc_L = fem.dirichletbc(u_L, dofs_L)
bc_R = fem.dirichletbc(u_R, dofs_R)
bcs = [bc_L, bc_R]

In [4]:
u = ufl.TrialFunction(V)
phi = ufl.TestFunction(V)

In [5]:
def Omega_0(x):
    return x[0] <= 0.5

def Omega_1(x):
    return x[0] >= 0.5

sigma = fem.Function(V)

cells_0 = mesh.locate_entities(Omega, Omega.topology.dim, Omega_0)
cells_1 = mesh.locate_entities(Omega, Omega.topology.dim, Omega_1)

sigma.x.array[cells_0] = np.full_like(cells_0, 1, dtype=default_scalar_type)
sigma.x.array[cells_1] = np.full_like(cells_1, 2, dtype=default_scalar_type)

In [6]:
a = ufl.inner(sigma * ufl.grad(u), ufl.grad(phi)) * ufl.dx
L = fem.Constant(Omega, default_scalar_type(0)) * phi * ufl.dx

In [7]:
problem = LinearProblem(a, L, bcs=bcs)#, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
u = problem.solve()

In [8]:
import dolfinx
with dolfinx.io.VTXWriter(MPI.COMM_WORLD, "./u_sol.bp", [u], engine="BP4") as vtx:
    vtx.write(0.0)

In [9]:
from dolfinx import io
with io.XDMFFile(Omega.comm, "u.xdmf", "w") as xdmf:
    xdmf.write_mesh(Omega)
    xdmf.write_function(u)